# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [46]:
df = pd.read_csv('dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [47]:
df.isna().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [48]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [49]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00


In [50]:
print("Number of unique IDs:", df['ID'].nunique())
print("Number of unique SystemCodeNumbers:", df['SystemCodeNumber'].nunique())


Number of unique IDs: 18368
Number of unique SystemCodeNumbers: 14


In [51]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['SystemCodeNumber_Encoded'] = label_encoder.fit_transform(df['SystemCodeNumber'])

print(df[['SystemCodeNumber', 'SystemCodeNumber_Encoded']])


      SystemCodeNumber  SystemCodeNumber_Encoded
0          BHMBCCMKT01                         0
1          BHMNCPHST01                         4
2          BHMMBMMBX01                         3
3          BHMNCPNST01                         5
4             Shopping                        13
...                ...                       ...
18363      BHMEURBRD01                         2
18364      BHMBCCTHL01                         1
18365      BHMBCCMKT01                         0
18366   Others-CCCPS98                        12
18367         Shopping                        13

[18368 rows x 2 columns]


In [52]:
label_encoder = LabelEncoder()
df['TrafficConditionNearby_Encoded'] = label_encoder.fit_transform(df['TrafficConditionNearby'])
df['VehicleType_Encoded'] = label_encoder.fit_transform(df['VehicleType'])

print(df[['TrafficConditionNearby', 'TrafficConditionNearby_Encoded']])
print(df[['VehicleType', 'VehicleType_Encoded']])

      TrafficConditionNearby  TrafficConditionNearby_Encoded
0                        low                               2
1                        low                               2
2                        low                               2
3                        low                               2
4                        low                               2
...                      ...                             ...
18363                    low                               2
18364                    low                               2
18365                    low                               2
18366                    low                               2
18367                    low                               2

[18368 rows x 2 columns]
      VehicleType  VehicleType_Encoded
0             car                    1
1            bike                    0
2             car                    1
3             car                    1
4           cycle                    2
...     

In [53]:
print(df[['VehicleType', 'VehicleType_Encoded']].drop_duplicates())

   VehicleType  VehicleType_Encoded
0          car                    1
1         bike                    0
4        cycle                    2
14       truck                    3


In [54]:
traffic = {'low': 0, 'average': 0.5, 'high': 1}
df['Traffic'] = df['TrafficConditionNearby'].map(traffic)

print(df[['TrafficConditionNearby', 'Traffic']].drop_duplicates())

   TrafficConditionNearby  Traffic
0                     low      0.0
7                 average      0.5
86                   high      1.0


In [55]:
print("Number of unique TrafficConditionNearby:", df['TrafficConditionNearby'].nunique())

Number of unique TrafficConditionNearby: 3


In [56]:

print("Unique values of special day:", df['IsSpecialDay'].unique())

Unique values of special day: [0 1]


In [57]:
VType = {'cycle': 0, 'bike': 0.33, 'car': 0.66, 'truck': 1}
df['VehicleTypeWeight'] = df['VehicleType'].map(VType)

print(df[['VehicleType', 'VehicleTypeWeight']].drop_duplicates())

   VehicleType  VehicleTypeWeight
0          car               0.66
1         bike               0.33
4        cycle               0.00
14       truck               1.00


In [58]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['QueueLength_Normalized'] = scaler.fit_transform(df[['QueueLength']])

print(df[['QueueLength', 'QueueLength_Normalized']])

       QueueLength  QueueLength_Normalized
0                1                0.066667
1                2                0.133333
2                2                0.133333
3                2                0.133333
4                2                0.133333
...            ...                     ...
18363            2                0.133333
18364            2                0.133333
18365            2                0.133333
18366            3                0.200000
18367            2                0.133333

[18368 rows x 2 columns]


In [59]:
df['Occ/Cap'] = df['Occupancy'] / df['Capacity']

print(df[['Occupancy', 'Capacity', 'Occ/Cap']])

       Occupancy  Capacity   Occ/Cap
0             61       577  0.105719
1            237      1200  0.197500
2            264       687  0.384279
3            249       485  0.513402
4            614      1920  0.319792
...          ...       ...       ...
18363        373       470  0.793617
18364        387       387  1.000000
18365        193       577  0.334489
18366       1671      3103  0.538511
18367       1180      1920  0.614583

[18368 rows x 3 columns]


In [60]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['Occ_Cap_Norm'] = scaler.fit_transform(df[['Occ/Cap']])

print(df[['Occ/Cap', 'Occ_Cap_Norm']])

        Occ/Cap  Occ_Cap_Norm
0      0.105719      0.098521
1      0.197500      0.186953
2      0.384279      0.366915
3      0.513402      0.491326
4      0.319792      0.304781
...         ...           ...
18363  0.793617      0.761314
18364  1.000000      0.960165
18365  0.334489      0.318942
18366  0.538511      0.515518
18367  0.614583      0.588814

[18368 rows x 2 columns]


In [61]:
df.VehicleTypeWeight

,VehicleTypeWeight
0,0.66
1,0.33
2,0.66
3,0.66
4,0.00
...,...
18363,0.66
18364,0.66
18365,0.00
18366,0.66


In [62]:
# Save the selected columns from the 'main' DataFrame to a CSV file for streaming or downstream processing
df[['SystemCodeNumber_Encoded','QueueLength_Normalized', 'VehicleTypeWeight', 'Traffic','IsSpecialDay','Occ_Cap_Norm','Timestamp']].to_csv('Parking_stream.csv', index=False)


In [63]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occ_Cap_Norm: float   # Number of occupied parking spots
    IsSpecialDay: int    # Total parking capacity at the location
    Traffic: float
    VehicleTypeWeight: float
    QueueLength_Normalized: float
    SystemCodeNumber_Encoded: int

In [64]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("Parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [65]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [66]:
import pathway as pw
import datetime

demand_window = (
    data_with_time.groupby(pw.this.SystemCodeNumber_Encoded)
    .windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        SystemCodeNumber_Encoded=pw.this._pw_instance,
        t=pw.this._pw_window_end,
        occ_norm=pw.reducers.avg(pw.this.Occ_Cap_Norm),
        queue_norm=pw.reducers.max(pw.this.QueueLength_Normalized),
        traffic_norm=pw.reducers.avg(pw.this.Traffic),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight=pw.reducers.avg(pw.this.VehicleTypeWeight)
    )
    .with_columns(
        base_price=10,
        demand_index=(
            0.3 * pw.this.occ_norm +
            0.3 * pw.this.queue_norm +
            0.2 * pw.this.traffic_norm +
            0.1 * pw.this.is_special_day +
            0.2 * pw.this.vehicle_weight
        ),
    )
    .with_columns(
        price=pw.this.base_price * (1 + pw.this.demand_index),
    )
    .with_columns(
        price_clamped=pw.apply(
            lambda x: max(5, min(20, x)),
            pw.this.price
        )
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [67]:
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price_clamped evolves over time
    fig.line("t", "price_clamped", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price_clamped", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = demand_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [68]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

In [69]:
main = pd.read_csv( "Parking_stream.csv")
main

,SystemCodeNumber_Encoded,QueueLength_Normalized,VehicleTypeWeight,Traffic,IsSpecialDay,Occ_Cap_Norm,Timestamp
0,0,0.066667,0.66,0.0,0,0.098521,2016-10-04 07:59:00
1,4,0.133333,0.33,0.0,0,0.186953,2016-10-04 07:59:00
2,3,0.133333,0.66,0.0,0,0.366915,2016-10-04 07:59:00
3,5,0.133333,0.66,0.0,0,0.491326,2016-10-04 07:59:00
4,13,0.133333,0.00,0.0,0,0.304781,2016-10-04 07:59:00
...,...,...,...,...,...,...,...
18363,2,0.133333,0.66,0.0,0,0.761314,2016-12-19 16:30:00
18364,1,0.133333,0.66,0.0,0,0.960165,2016-12-19 16:30:00
18365,0,0.133333,0.00,0.0,0,0.318942,2016-12-19 16:30:00
18366,12,0.200000,0.66,0.0,0,0.515518,2016-12-19 16:30:00


In [70]:
print(main.columns)

Index(['SystemCodeNumber_Encoded', 'QueueLength_Normalized',
       'VehicleTypeWeight', 'Traffic', 'IsSpecialDay', 'Occ_Cap_Norm',
       'Timestamp'],
      dtype='object')


In [71]:
main['demand_index'] = (
    0.3 * main['Occ_Cap_Norm'] +
    0.3 * main['QueueLength_Normalized'] +
    0.2 * main['Traffic'] +
    0.1 * main['IsSpecialDay'] +
    0.2 * main['VehicleTypeWeight']
)
main['price'] = 10 * (1 + main['demand_index'])
main['price_clamped'] = main['price'].clip(lower=5, upper=20)


In [72]:
main['Timestamp'] = pd.to_datetime(main['Timestamp'])


In [73]:
import panel as pn
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

pn.extension('bokeh')

station_ids = main['SystemCodeNumber_Encoded'].unique().tolist()
station_selector = pn.widgets.Select(options=station_ids, name='Station ID')

@pn.depends(station_selector)
def price_plotter_interactive(station_id):
    data = main[main['SystemCodeNumber_Encoded'] == station_id]
    if data.empty:
        return pn.pane.Markdown("No data for this station.", height=400)
    source = ColumnDataSource(data)
    p = figure(
        height=400,
        width=800,
        title=f"Daily Clamped Price for Station {station_id}",
        x_axis_type="datetime"
    )
    p.line("Timestamp", "price_clamped", source=source, line_width=2, color="navy")
    p.scatter("Timestamp", "price_clamped", source=source, size=6, color="red", marker="circle")
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Clamped Price"
    return p

pn.Column(station_selector, price_plotter_interactive).servable()


Column
    [0] Select(name='Station ID', options=[0, 4, 3, 5, 13, ...], value=0)
    [1] ParamFunction(function, _pane=Bokeh, defer_load=False)